In [1]:
import os
import requests
import pandas as pd
import sqlite3

In [2]:
# download dos arquivos

# URL base para os Datasets do IMDb

base_url = "https://datasets.imdbws.com"

# lista de nomes dos arquivos que deseja baixar

arquivos = [
    "name.basics.tsv.gz",
    "title.akas.tsv.gz",
    "title.basics.tsv.gz",
    "title.crew.tsv.gz",
    "title.episode.tsv.gz",
    "title.principals.tsv.gz",
    "title.ratings.tsv.gz"
]

# Diretório de destino
destino_diretorio = "data"

# Checando se o diretório de destino existe

os.makedirs(destino_diretorio, exist_ok=True)

# Loop para baixar cada arquivo

for arquivo in arquivos:
    url = base_url + "/" + arquivo

    caminho_destino = os.path.join(destino_diretorio , arquivo)

    # Verifica se o arquivo já existe para evitar download repetido
    if not os.path.exists(caminho_destino):
        print(f"Baixando {arquivo}...")
        response = requests.get(url)

        # Verifica se a solicitação foi bem sucedida (Código de status 200)
        if response.status_code == 200:
            with open(caminho_destino, 'wb') as f:
                f.write(response.content)
                print(f"{arquivo} baixado com sucesso")
        else:
            print(f"Falha ao baixar {arquivo}. Código de status: {response.status_code}")

    else:
        print(f"{arquivo} já existe. Pulando download.")

print("Download Concluído.")

Baixando name.basics.tsv.gz...
name.basics.tsv.gz baixado com sucesso
Baixando title.akas.tsv.gz...
title.akas.tsv.gz baixado com sucesso
Baixando title.basics.tsv.gz...
title.basics.tsv.gz baixado com sucesso
Baixando title.crew.tsv.gz...
title.crew.tsv.gz baixado com sucesso
Baixando title.episode.tsv.gz...
title.episode.tsv.gz baixado com sucesso
Baixando title.principals.tsv.gz...
title.principals.tsv.gz baixado com sucesso
Baixando title.ratings.tsv.gz...
title.ratings.tsv.gz baixado com sucesso
Download Concluído.


In [3]:
# Tratamento dos dados

# Diretórios

diretorio_dados = "data"

diretorio_tratados = os.path.join(diretorio_dados, "tratados")

# Garante que o diretório "tratados" existe

os.makedirs(diretorio_tratados, exist_ok=True)

# Lista todos os arquivos no diretório "data"

arquivos= os.listdir(diretorio_dados)

# Loop para abrir, tratar e salvar cada arquivo

for arquivo in arquivos:
    caminho_arquivo=os.path.join(diretorio_dados, arquivo)

    if os.path.isfile(caminho_arquivo) and arquivo.endswith("gz"):
        print(f"Lendo e tratando arquivo {arquivo}...")
#Lê o arquivo TSV usando pandas
        df=pd.read_csv(caminho_arquivo, sep='\t', compression='gzip', low_memory=False)

# Substitui os caracteres '\n' por um valor nulo
        df.replace({"\\N": None}, inplace=True)

# Salva o DataFrame no diretório "Tratados" sem compressão

        caminho_destino= os.path.join(diretorio_tratados, arquivo [:-3]) # Remove a extensão . gz
        df.to_csv(caminho_destino, sep='\t', index=False)

        print(f"Tratamento concluído para {arquivo}. Arquivo tratado salvo em {caminho_destino}")
print("Todos os arquivos foram tratados e salvos no diretório 'tratados'.")

Lendo e tratando arquivo name.basics.tsv.gz...
Tratamento concluído para name.basics.tsv.gz. Arquivo tratado salvo em data\tratados\name.basics.tsv
Lendo e tratando arquivo title.akas.tsv.gz...
Tratamento concluído para title.akas.tsv.gz. Arquivo tratado salvo em data\tratados\title.akas.tsv
Lendo e tratando arquivo title.basics.tsv.gz...
Tratamento concluído para title.basics.tsv.gz. Arquivo tratado salvo em data\tratados\title.basics.tsv
Lendo e tratando arquivo title.crew.tsv.gz...
Tratamento concluído para title.crew.tsv.gz. Arquivo tratado salvo em data\tratados\title.crew.tsv
Lendo e tratando arquivo title.episode.tsv.gz...
Tratamento concluído para title.episode.tsv.gz. Arquivo tratado salvo em data\tratados\title.episode.tsv
Lendo e tratando arquivo title.principals.tsv.gz...
Tratamento concluído para title.principals.tsv.gz. Arquivo tratado salvo em data\tratados\title.principals.tsv
Lendo e tratando arquivo title.ratings.tsv.gz...
Tratamento concluído para title.ratings.tsv.g

In [2]:
# ==============================
# VALIDANDO DATAFRAME (AMOSTRA)
# ==============================

df = pd.read_csv(
    "./data/tratados/name.basics.tsv",
    sep="\t",
    nrows=10_000  # lê só uma amostra para validação
)

df.head()

# Verifica a quantidade de dados nulos por coluna
df.isnull().sum()


# ==============================
# SALVANDO EM BANCO DE DADOS SQLITE
# ==============================

# Diretórios
diretorio_tratados = os.path.join("data", "tratados")
banco_dados = "imdb_data.db"

# Conecta ao banco de dados SQLite
conexao = sqlite3.connect(banco_dados)

# Lista todos os arquivos no diretório "tratados"
arquivos = os.listdir(diretorio_tratados)

# Tamanho do chunk para evitar estouro de memória
chunksize = 100_000





In [3]:
# ==============================
# LOOP PARA LER E SALVAR OS ARQUIVOS
# ==============================

for arquivo in arquivos:
    caminho_arquivo = os.path.join(diretorio_tratados, arquivo)

    if os.path.isfile(caminho_arquivo) and arquivo.endswith(".tsv"):

        print(f"Processando {arquivo}...")

        # Remove a extensão do nome do arquivo
        nome_tabela = os.path.splitext(arquivo)[0]

        # Substitui caracteres especiais no nome da tabela
        nome_tabela = nome_tabela.replace(".", "_").replace("-", "_")

        # Remove a tabela se já existir
        conexao.execute(f"DROP TABLE IF EXISTS {nome_tabela}")

        # Leitura do arquivo em chunks
        for chunk in pd.read_csv(
            caminho_arquivo,
            sep="\t",
            chunksize=chunksize,
            low_memory=False
        ):
            # Salva cada chunk no SQLite
            chunk.to_sql(
                nome_tabela,
                conexao,
                index=False,
                if_exists="append"
            )

        print(f"Tabela {nome_tabela} salva com sucesso.")

# Fecha conexão com o banco de dados
conexao.close()

print("Todos os arquivos foram salvos no banco de dados.")

Processando name.basics.tsv...
Tabela name_basics salva com sucesso.
Processando title.akas.tsv...
Tabela title_akas salva com sucesso.
Processando title.basics.tsv...
Tabela title_basics salva com sucesso.
Processando title.crew.tsv...
Tabela title_crew salva com sucesso.
Processando title.episode.tsv...
Tabela title_episode salva com sucesso.
Processando title.principals.tsv...
Tabela title_principals salva com sucesso.
Processando title.ratings.tsv...
Tabela title_ratings salva com sucesso.
Todos os arquivos foram salvos no banco de dados.


In [3]:
# Nome do banco de dados

banco_dados = "imdb_data.db"

# Conecta-se ao Banco de dados SQLite

conexao = sqlite3.connect(banco_dados)

# Cria um cursor

cursor = conexao.cursor()

# Executa a consulta SQL para obter o nome das tabelas

cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ;")

# Recupera os resultados da consulta

tabelas = cursor.fetchall()

# Exibe o nome das tabelas

print ("Tabelas no banco de dados:")

for tabela in tabelas:
    print(tabela[0])

# Fecha cursor e a conexão com banco de dados

cursor.close()

conexao.close()



Tabelas no banco de dados:
name_basics
title_akas
title_basics
title_crew
title_episode
title_principals
title_ratings
analitico_titulo


In [4]:
import shutil

# Banco de dados na mesma pasta do script
banco_dados = "imdb_data.db"

# Pasta temporária local segura
temp_dir = os.path.join(os.getcwd(), "temp_sqlite")
os.makedirs(temp_dir, exist_ok=True)

# Define o diretório temporário do SQLite
os.environ["TMPDIR"] = temp_dir
os.environ["SQLITE_TMPDIR"] = temp_dir

# Verifica espaço livre (precisa de pelo menos 200 MB)
total, used, free = shutil.disk_usage(os.getcwd())
if free < 200 * 1024 * 1024:
    raise OSError("Espaço insuficiente em disco. Libere ao menos 200MB e tente novamente.")

# Conecta ao banco
conexao = sqlite3.connect(banco_dados)

try:
    # Força o SQLite a usar a pasta temporária criada
    conexao.execute(f"PRAGMA temp_store_directory='{temp_dir}';")

    query = """
    SELECT 
        tconst,
        COUNT(DISTINCT nconst) AS qtParticipantes
    FROM title_principals
    GROUP BY tconst
    LIMIT 10;
    """

    df = pd.read_sql_query(query, conexao)

except sqlite3.OperationalError as e:
    if "database or disk is full" in str(e).lower():
        print("Erro: o disco ou diretório temporário está cheio ou sem permissão de escrita.")
        print(f"Tente liberar espaço e limpe a pasta temporária: {temp_dir}")
    else:
        raise
finally:
    conexao.close()


In [5]:
df

,tconst,qtParticipantes
0,tt0000001,3
1,tt0000002,2
2,tt0000003,4
3,tt0000004,2
4,tt0000005,3
5,tt0000007,5
6,tt0000008,3
7,tt0000009,4
8,tt0000010,1
9,tt0000011,2


In [28]:
import sqlite3
import pandas as pd

# Banco na mesma pasta
banco_dados = "imdb_data.db"

# Conecta ao banco
conexao = sqlite3.connect(banco_dados)

# Query com agregação de participantes
query = """
SELECT 
     tb.tconst,
     tb.titleType,
     tb.originalTitle,
     tb.endYear,
     tb.genres,
     tr.averageRating,
     tr.numVotes,
     tp.qtParticipantes
FROM title_basics tb

LEFT JOIN title_ratings tr
    ON tr.tconst = tb.tconst

LEFT JOIN (
    SELECT 
        tconst,
        COUNT(DISTINCT nconst) AS qtParticipantes
    FROM title_principals
    GROUP BY tconst
) AS tp
    ON tp.tconst = tb.tconst

LIMIT 10;
"""

# Executa a consulta
df = pd.read_sql_query(query, conexao)

# Fecha a conexão
conexao.close()

# Mostra o resultado
print(df)


      tconst titleType                                originalTitle endYear  \
0  tt0000001     short                                   Carmencita    None   
1  tt0000002     short                       Le clown et ses chiens    None   
2  tt0000003     short                               Pauvre Pierrot    None   
3  tt0000004     short                                  Un bon bock    None   
4  tt0000005     short                             Blacksmith Scene    None   
5  tt0000006     short                            Chinese Opium Den    None   
6  tt0000007     short  Corbett and Courtney Before the Kinetograph    None   
7  tt0000008     short       Edison Kinetoscopic Record of a Sneeze    None   
8  tt0000009     movie                                   Miss Jerry    None   
9  tt0000010     short          La sortie de l'usine Lumière à Lyon    None   

                     genres  averageRating  numVotes  qtParticipantes  
0         Documentary,Short            5.7      2188      

In [35]:
analitico_titulo = """
CREATE TABLE IF NOT EXISTS analitico_titulo AS
WITH participantes AS (
    SELECT 
        tb.tconst,
        tb.titleType,
        tb.originalTitle,
        tb.endYear,
        tb.genres,
        tr.averageRating,
        tr.numVotes,
        tp.qtParticipantes
    FROM title_basics tb
    LEFT JOIN title_ratings tr
        ON tr.tconst = tb.tconst
    LEFT JOIN (
        SELECT 
            tconst,
            COUNT(DISTINCT nconst) AS qtParticipantes
        FROM title_principals
        GROUP BY tconst
    ) AS tp
        ON tp.tconst = tb.tconst
)
SELECT * FROM participantes;
"""

analitico_participante = """
SELECT
    tp.nconst,
    tp.tconst,
    tp.ordering,
    tp.category,
    tb.genres
FROM title_principals tp
LEFT JOIN title_basics tb
    ON tb.tconst = tp.tconst;
"""


In [36]:
# Lista de consultas
queries = [analitico_titulo, analitico_participante]

for query in queries:
    # Diretórios
    banco_dados = "imdb_data.db"

    # Conecta ao banco de dados SQLite
    conexao = sqlite3.connect(banco_dados)

    # Executa a consulta SQL
    conexao.execute(query)

    # Fecha a conexão com o banco de dados
    conexao.close()

print("Tabelas criadas com sucesso.")


Tabelas criadas com sucesso.
